In [ ]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint

# Fleet optimisation and charging infrastructure dimensioning
## Dynamic programming approach
We use the demand at each hour for dimensioning the electric fleet and the charging infrastructure.
- We assume that we can choose only one kind of vehicle 
- We assume that we don't have any budget constrain
- We assume that we don't have any maximum power constrain

### Parameter introduction and initialisation

In [1]:
import numpy as np
from scipy.optimize import minimize, LinearConstraint, SR1

rng = np.random.default_rng(seed=40) #seed for reproducible results
demand=[10,10,10,12,12,12,10,11,11,10,3,10,10,10,10,10,10,10,4,6,6,4,10,7,10,10,10,10,10,10,4,6,6,4,10,10,10,10,10,10,10,10,4,6,6,4,10,7]
T=len(demand)
battery_capacity=112 #kWh SOC=100%
charging_powers=np.array([7.4, 20, 100, 350])
r_s=[1/(battery_capacity/charging_powers[0])*100, 1/(0.95*battery_capacity/charging_powers[1]+0.5)*100, 1/(0.95*battery_capacity/charging_powers[2]+0.5)*100, 1/(0.95*battery_capacity/charging_powers[3]+0.5)*100] #recharge rate 1/h
investment_cost=100000 #euros
electricity_cost=np.array([1, 3, 5, 8])
power_per_hour=14.43 #kW/h
d_s=1/(battery_capacity/power_per_hour)*100 #discharge rate 1/h
E_diss=0 #energy consumption in operation
E_abs=0 #energy used in charge
f=[0, 0, 0, 0] #counter of stations for each charging power
v_t=demand[0] #initialization of n. min of vehicles to satisfy the demand of the first time
x=np.zeros((v_t,len(charging_powers))) #decision variable, vehicles per station #, dtype=bool_
threshold=20 #SOC % for safety reason
SOC=np.zeros((v_t, 2))
charging=0

for j in range(v_t):
    SOC[j][0]=rng.integers(low=50, high=100, size=1) #initial SOC randomly chosen
    
E_abs=sum(SOC[:,0])# for j in range(v_t))
cost=np.zeros(T)

def cost_function(x, delta_SOC, electricity_cost, investment_cost, v_t):
    re_delta=np.tile(delta_SOC, len(electricity_cost))
    re_elt=np.stack([electricity_cost[0].repeat(len(delta_SOC)),electricity_cost[1].repeat(len(delta_SOC)),electricity_cost[2].repeat(len(delta_SOC)),electricity_cost[3].repeat(len(delta_SOC))]).reshape(-1)
    return investment_cost*v_t + sum(re_delta* x * re_elt) #objective function #.reshape(-1)

# def cost_function(x, delta_SOC, electricity_cost, investment_cost, v_t):
# #     re_delta=np.tile(delta_SOC, len(electricity_cost))
# #     re_elt=np.stack([electricity_cost[0].repeat(len(delta_SOC)),electricity_cost[1].repeat(len(delta_SOC)),electricity_cost[2].repeat(len(delta_SOC)),electricity_cost[3].repeat(len(delta_SOC))]).reshape(-1)
#     return investment_cost*v_t + np.dot(np.dot(delta_SOC,x), electricity_cost) #objective function

# def x_number(x, charging):
#     return sum(sum(x))-charging #the active variables must be equal to the n. of vehicles that need charging

# def x_pow(x):
#     return sum(x[:,1])-1  #I can not charge one vehicle with more than one mode

# def x_bool(x): 
#     pass
cons = [{'type': 'eq', 'fun': lambda x: np.sum(x.reshape(-1))-charging}, #not respected, why?
        {'type': 'ineq', 'fun': lambda x: 1-np.sum(x[0])},
        {'type': 'ineq', 'fun': lambda x: 1-np.sum(x[1])},
        {'type': 'ineq', 'fun': lambda x: 1-np.sum(x[2])},
        {'type': 'ineq', 'fun': lambda x: 1-np.sum(x[3])}]

delta_SOC = np.zeros(v_t)
#cons=[{'type':'eq', 'fun': sum(sum(x))-charging}, {'type':'eq', 'fun': sum(x[:,1])-1},]
#argss=(x, delta_SOC, electricity_cost, investment_cost, v_t)

In [2]:
x

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [3]:
tot_cost=0
for t in range(T): #for each hour
    
    x_0 = x#.reshape(-1)
    
    res = minimize(
        cost_function, 
        #method='trust-constr',
        #jac='2-point', 
        #hess=SR1(),
        x0=x_0, 
        args=(delta_SOC, electricity_cost, investment_cost, v_t), 
        constraints=cons
    )
    
    cost[t]=np.float64(res.fun)-investment_cost*v_t
    
    for i in range(len(electricity_cost)):
        f[i] = max(f[i], sum(x[j][i] for j in range(v_t)))
    
    tot_cost= tot_cost+cost[t]
    #counters to zero
    delta_SOC = np.zeros(v_t)
    delta_SOC_pos = np.zeros(v_t)
    i=0
    j=0
    
    for j in range(v_t): #for each vehicle I check
        
        if SOC[j][0] < 100 and SOC[j][1] == 1:
            #charging process
            en=np.dot(x[j,:], r_s)
            SOC[j][0] = SOC[j][0] + en
            E_abs = E_abs+en
            delta_SOC[j] = en
            print(SOC)
            
        elif SOC[j][0] <= threshold and SOC[j][1] == 0: 
            #I start the charging process
            SOC[j][1] = 1
        
        elif SOC[j][0] >= threshold and SOC[j][0] <= 100 and SOC[j][1] == 0:
            #Using the vehicle
            SOC[j][0]=SOC[j][0]-d_s
            E_diss=E_diss+d_s
            delta_SOC_pos[j] = d_s
        
        
            
        elif SOC[j][0] >= 100 and SOC[j][1] == 1:
            #stop the charging process
            SOC[j][1] = 0
            SOC[j][0] = 100
        
        elif SOC[j][0] >= 100 and SOC[j][1] == 0:
            SOC[j][1] = 0
            SOC[j][0] = 100
            
    charging=np.int32(sum(SOC[j][1] for j in range(v_t)))
    
    if v_t-charging < demand[t]: #Guarantee the satisfaction of the demand
        n_new_vehicles = np.int32(demand[t] - (v_t - charging))
        v_t = v_t + n_new_vehicles
        new_vehicle = np.array([100, 0])
        addendum = np.tile(new_vehicle, (n_new_vehicles,1))
        SOC = np.append(SOC,addendum,axis=0)
        delta_SOC = np.append(delta_SOC,np.zeros(n_new_vehicles),axis=0)
        #delta_SOC + 0*n_new_vehicles
        new = np.tile([0,0,0,0], (n_new_vehicles,1))
        x = np.vstack([x,new])
    
    #print(delta_SOC)

    print(SOC)
    
if E_abs<=E_diss:
    print("Error, energy balance not respected")
    
tot_cost=tot_cost+investment_cost*v_t
print(cost, tot_cost, f)

/var/folders/x8/typb4bcj1d978dqysxq0mc0r0000gn/T/ipykernel_83323/1621615624.py:6: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  res = minimize(


ValueError: `x0` must have at most 1 dimension.